In [2]:
#Import requirements:
import cv2 as cv


ModuleNotFoundError: No module named 'cv2'

# The Pong Project

### Répartition des tâches:
Nous avons décidé de séparer le projet en 2 parties: 
1. le jeu (avec comme controles les "UP ARROW" "DOWN ARROW" normaux, fait par Christian et Aissata)
2. les contrôles fait par une main, grâce à OpenCV : fait par Paul & Raphaela
Puis nous allons intégrer ces 2 parties ensemble

De plus, afin d'apprendre à utiliser git (un des outils les plus importants dans notre carrière de développeurs) nous allons tout faire sur GitHub en essayant d'adopter les bonnes pratiques, comme créer une branche pour chaque feature avant de merge avec main.
